In [228]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [229]:
%matplotlib inline

In [230]:
import pandas as pd
import numpy as np

In [231]:
import bclm

In [232]:
dropped = [5438, 5444, 5445, 5446, 5448, 5449, 5450, 5451, 5453, 5459]
spdf = bclm.read_dataframe('spmrl')

In [233]:
global_dropped = [spdf[spdf.sent_id==d].global_sent_id.iat[0] for d in dropped]
global_dropped

[5438, 5444, 5445, 5446, 5448, 5449, 5450, 5451, 5453, 5459]

In [234]:
uddf = bclm.read_dataframe('ud')

In [235]:
uddf = uddf[(~uddf.global_sent_id.isin(global_dropped))]

In [236]:
from collections import OrderedDict

In [237]:
uddf.head().T

,0,1,2,3,4
sent_id,1,1,1,1,1
id,1,2,3,4,5
form,עשרות,אנשים,מגיעים,מ,תאילנד
lemma,עשרות,איש,הגיע,מ,תאילנד
upostag,NUM,NOUN,VERB,ADP,PROPN
xpostag,NUM,NOUN,VERB,ADP,PROPN
feats,"OrderedDict([('Definite', 'Cons'), ('Gender', ...","OrderedDict([('Gender', 'Masc'), ('Number', 'P...","OrderedDict([('Gender', 'Masc'), ('HebBinyan',...",NaN,NaN
head,2,3,0,5,3
deprel,nummod,nsubj,root,case,obl
deps,NaN,NaN,NaN,NaN,NaN


In [238]:
ud_sents = list(uddf.sent_id.unique())

## Lattices

In [239]:
import sys
sys.path.insert(0, '/home/nlp/danb/conllu')

In [240]:
import conllu
conllu.__file__

'/home/nlp/danb/conllu/conllu/__init__.py'

In [241]:
from conllu import parse
import bz2

In [242]:
import re
sent_id_re = re.compile('# sent_id = (\d+)')
sent_id_re.match('# sent_id = 123').group(1)

'123'

In [243]:
labels = ['O']
for b in ['B', 'I', 'S', 'E']:
    for t in ['PER', 'ORG', 'GPE', 'FAC', 'LOC', 'DUC', 'EVE', 'WOA', 'ANG']:
        labels.append(b+'-'+t)
                      
len(labels)

37

In [244]:
plo_labels = ['O']
for b in ['B', 'I', 'S', 'E']:
    for t in ['PER', 'ORG', 'LOC']:
        plo_labels.append(b+'-'+t)
                      
plo_labels

['O',
 'B-PER',
 'B-ORG',
 'B-LOC',
 'I-PER',
 'I-ORG',
 'I-LOC',
 'S-PER',
 'S-ORG',
 'S-LOC',
 'E-PER',
 'E-ORG',
 'E-LOC']

In [245]:
biose_labels = ['O']
for b in ['B', 'I', 'S', 'E']:
    for t in ['ENT']:
        biose_labels.append(b+'-'+t)
                      
biose_labels

['O', 'B-ENT', 'I-ENT', 'S-ENT', 'E-ENT']

In [246]:
import re

def add_feat(feats, name, value):
    feat = name+'='+value
    if feats=='_':
        feats = feat
    else:
        feats += '|' + feat
    
    return feats
sent_id_re = re.compile('# sent_id = (\d+)')
sent_ids = []
def read_ul(f, output_f, labels, add=True, field_name='biose_layer0'):
    sent_id='XXX'
    for line in f:
        line = line.strip()
        
        if sent_id in ud_sents and line=='':
                output_f.write('\n')
        elif line!='' and line[0]=='#':
            sid = sent_id_re.match(line)
            if sid:
                sent_id =  int(sid.group(1))
                rows = set()

            sent_ids.append(sent_id)
            if sent_id in ud_sents:
                output_f.write(line+'\n')
            else:
                print(line)
        elif sent_id in ud_sents:
                parts = line.split('\t')
                if '-' in parts[0]:
                    output_f.write(line+'\n')
                else:
                    ID1, ID2, form, lemma, upostag, xpostag, feats, token_id, x = parts
                    for label in labels:
                        if add: 
                            new_feats = add_feat(feats, field_name, label)
                        else:
                            new_feats = field_name+'='+label
                        new_row = '\t'.join([ID1, ID2, form, lemma, upostag, xpostag, new_feats, token_id, x])
                        if add or (not new_row in rows):
                            rows.add(new_row)
                            output_f.write(new_row+'\n')
            

In [247]:
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-dev.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/he_htb-ud-dev.heblex.ner_plo.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul(f, of, plo_labels, field_name='biose_layer0_plo')
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-dev.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/he_htb-ud-dev.heblex.ner_nocat.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul(f, of, biose_labels, field_name='biose_layer0_nocat')
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-dev.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/he_htb-ud-dev.heblex.ner_full.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul(f, of, labels, field_name='biose_layer0')

In [248]:
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-test.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/he_htb-ud-test.heblex.ner_plo.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul(f, of, plo_labels, field_name='biose_layer0_plo')
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-test.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/he_htb-ud-test.heblex.ner_nocat.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul(f, of, biose_labels, field_name='biose_layer0_nocat')
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-test.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/he_htb-ud-test.heblex.ner_full.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul(f, of, labels, field_name='biose_layer0')

In [249]:
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-train.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/he_htb-ud-train.heblex.ner_plo.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul(f, of, plo_labels, field_name='biose_layer0_plo')
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-train.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/he_htb-ud-train.heblex.ner_nocat.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul(f, of, biose_labels, field_name='biose_layer0_nocat')
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-train.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/he_htb-ud-train.heblex.ner_full.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul(f, of, labels, field_name='biose_layer0')


# sent_id = 1616
# text = לונדון (רויטר).
# sent_id = 2405
# text = לונדון (מיוחד ל"הארץ").
# sent_id = 2837
# text = מדוע?
# sent_id = 3301
# text = לא.
# sent_id = 3314
# text = (ע).
# sent_id = 3379
# text = ואשינגטון (מיוחד ל"הארץ").
# sent_id = 3549
# text = לונדון (מיוחד ל"הארץ").
# sent_id = 3749
# text = (ע).
# sent_id = 3785
# text = לונדון (מיוחד ל"הארץ").
# sent_id = 4058
# text = כן.
# sent_id = 4069
# text = לא.
# sent_id = 4210
# text = (ע).
# sent_id = 4240
# text = ?
# sent_id = 4829
# text = מבלבלים לכם את המוח, הממשלה לא תיפול, השיב לוי.
# sent_id = 5071
# text = (ע).
# sent_id = 5099
# text = (ע).
# sent_id = 5164
# text = בהחלט.
# sent_id = 5169
# text = ואשינגטון (מיוחד ל"הארץ").
# sent_id = 5282
# text = ואשינגטון (מיוחד ל"הארץ").
# sent_id = 5432
# text = בוקר אחד, לפני כחמישה חודשים, התייצבו סרגיי, ווליה, מישה ושתי מזוודות על הדשא של קיבוץ רוחמה.
# sent_id = 5433
# text = סרגיי דוברוביצקי, רופא קרדיולוג בעל ותק של 11 שנים בבריה"ם, נתון עתה בעיצומן של בחינות בבתי

### remove other feats

In [250]:
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-dev.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/he_htb-ud-dev.heblex.ner_plo_only.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul(f, of, plo_labels, add=False, field_name='biose_layer0_plo')
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-dev.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/he_htb-ud-dev.heblex.ner_nocat_only.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul(f, of, biose_labels, add=False, field_name='biose_layer0_nocat')
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-dev.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/he_htb-ud-dev.heblex.ner_full_only.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul(f, of, labels, add=False, field_name='biose_layer0')

In [251]:
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-test.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/he_htb-ud-test.heblex.ner_plo_only.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul(f, of, plo_labels, add=False, field_name='biose_layer0_plo')
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-test.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/he_htb-ud-test.heblex.ner_nocat_only.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul(f, of, biose_labels, add=False, field_name='biose_layer0_nocat')
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-test.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/he_htb-ud-test.heblex.ner_full_only.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul(f, of, labels, add=False, field_name='biose_layer0')

In [252]:
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-train.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/he_htb-ud-train.heblex.ner_plo_only.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul(f, of, plo_labels, add=False, field_name='biose_layer0_plo')
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-train.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/he_htb-ud-train.heblex.ner_nocat_only.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul(f, of, biose_labels, add=False, field_name='biose_layer0_nocat')
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-train.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/he_htb-ud-train.heblex.ner_full_only.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul(f, of, labels, add=False, field_name='biose_layer0')


# sent_id = 1616
# text = לונדון (רויטר).
# sent_id = 2405
# text = לונדון (מיוחד ל"הארץ").
# sent_id = 2837
# text = מדוע?
# sent_id = 3301
# text = לא.
# sent_id = 3314
# text = (ע).
# sent_id = 3379
# text = ואשינגטון (מיוחד ל"הארץ").
# sent_id = 3549
# text = לונדון (מיוחד ל"הארץ").
# sent_id = 3749
# text = (ע).
# sent_id = 3785
# text = לונדון (מיוחד ל"הארץ").
# sent_id = 4058
# text = כן.
# sent_id = 4069
# text = לא.
# sent_id = 4210
# text = (ע).
# sent_id = 4240
# text = ?
# sent_id = 4829
# text = מבלבלים לכם את המוח, הממשלה לא תיפול, השיב לוי.
# sent_id = 5071
# text = (ע).
# sent_id = 5099
# text = (ע).
# sent_id = 5164
# text = בהחלט.
# sent_id = 5169
# text = ואשינגטון (מיוחד ל"הארץ").
# sent_id = 5282
# text = ואשינגטון (מיוחד ל"הארץ").
# sent_id = 5432
# text = בוקר אחד, לפני כחמישה חודשים, התייצבו סרגיי, ווליה, מישה ושתי מזוודות על הדשא של קיבוץ רוחמה.
# sent_id = 5433
# text = סרגיי דוברוביצקי, רופא קרדיולוג בעל ותק של 11 שנים בבריה"ם, נתון עתה בעיצומן של בחינות בבתי

## Replace POS

In [296]:
import re

sent_id_re = re.compile('# sent_id = (\d+)')
sent_ids = []
def read_ul_replace_pos(f, output_f, labels, field_name='cat'):
    sent_id='XXX'
    for line in f:
        line = line.strip()
        
        if sent_id in ud_sents and line=='':
                output_f.write('\n')
        elif line!='' and line[0]=='#':
            sid = sent_id_re.match(line)
            if sid:
                sent_id =  int(sid.group(1))
                rows = set()
#             if sent_id>2:
#                 break
            sent_ids.append(sent_id)
            if sent_id in ud_sents:
                output_f.write(line+'\n')
            else:
                print(line)
                
        elif sent_id in ud_sents:
                parts = line.split('\t')
                if '-' in parts[0]:
                    output_f.write(line+'\n')
                else:
                    ID1, ID2, form, lemma, upostag, xpostag, feats, token_id, x = parts
                    for label in labels:
                        if label=='O':
                            bio='O'
                            new_feats = '_'
                        else:
                            bio, cat = label.split('-')
                            new_feats = field_name+'='+cat
                            
                        new_pos = bio
                        
                        new_row = '\t'.join([ID1, ID2, form, lemma, new_pos, new_pos, new_feats, token_id, x])
                        if not new_row in rows:
                            rows.add(new_row)
                            output_f.write(new_row+'\n')

In [297]:
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-dev.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/lattices_pos/he_htb-ud-dev.heblex.ner_pos_plo.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul_replace_pos(f, of, plo_labels, field_name='ner_cat_plo')
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-dev.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/lattices_pos/he_htb-ud-dev.heblex.ner_pos_nocat.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul_replace_pos(f, of, biose_labels, field_name='ner_cat_nocat')
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-dev.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/lattices_pos/he_htb-ud-dev.heblex.ner_pos_full.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul_replace_pos(f, of, labels, field_name='ner_cat_full')

In [298]:
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-test.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/lattices_pos/he_htb-ud-test.heblex.ner_pos_plo.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul_replace_pos(f, of, plo_labels, field_name='ner_cat_plo')
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-test.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/lattices_pos/he_htb-ud-test.heblex.ner_pos_nocat.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul_replace_pos(f, of, biose_labels, field_name='ner_cat_nocat')
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-test.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/lattices_pos/he_htb-ud-test.heblex.ner_pos_full.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul_replace_pos(f, of, labels, field_name='ner_cat_full')

In [299]:
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-train.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/lattices_pos/he_htb-ud-train.heblex.ner_pos_plo.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul_replace_pos(f, of, plo_labels, field_name='ner_cat_plo')
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-train.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/lattices_pos/he_htb-ud-train.heblex.ner_pos_nocat.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul_replace_pos(f, of, biose_labels, field_name='ner_cat_nocat')
with bz2.open('../UL_Hebrew-HTB/he_htb-ud-train.heblex.conllul.bz2', 'rt', encoding='utf-8') as f, open('data/for_amit/lattices_pos/he_htb-ud-train.heblex.ner_pos_full.conllul', 'w', encoding='utf-8') as of:
    ul_dev = read_ul_replace_pos(f, of, labels, field_name='ner_cat_full')


# sent_id = 1616
# text = לונדון (רויטר).
# sent_id = 2405
# text = לונדון (מיוחד ל"הארץ").
# sent_id = 2837
# text = מדוע?
# sent_id = 3301
# text = לא.
# sent_id = 3314
# text = (ע).
# sent_id = 3379
# text = ואשינגטון (מיוחד ל"הארץ").
# sent_id = 3549
# text = לונדון (מיוחד ל"הארץ").
# sent_id = 3749
# text = (ע).
# sent_id = 3785
# text = לונדון (מיוחד ל"הארץ").
# sent_id = 4058
# text = כן.
# sent_id = 4069
# text = לא.
# sent_id = 4210
# text = (ע).
# sent_id = 4240
# text = ?
# sent_id = 4829
# text = מבלבלים לכם את המוח, הממשלה לא תיפול, השיב לוי.
# sent_id = 5071
# text = (ע).
# sent_id = 5099
# text = (ע).
# sent_id = 5164
# text = בהחלט.
# sent_id = 5169
# text = ואשינגטון (מיוחד ל"הארץ").
# sent_id = 5282
# text = ואשינגטון (מיוחד ל"הארץ").
# sent_id = 5432
# text = בוקר אחד, לפני כחמישה חודשים, התייצבו סרגיי, ווליה, מישה ושתי מזוודות על הדשא של קיבוץ רוחמה.
# sent_id = 5433
# text = סרגיי דוברוביצקי, רופא קרדיולוג בעל ותק של 11 שנים בבריה"ם, נתון עתה בעיצומן של בחינות בבתי

## Make this parallel to gold

In [263]:
def get_string_feats(x):
    if x:
        return '|'.join([k+'='+v for k,v in eval(x).items()])
    else:
        return ''
                        
uddf['feats'] = uddf.feats.fillna('').apply(get_string_feats)

In [264]:
uddf['misc'] = uddf.misc.fillna('').apply(get_string_feats)

In [265]:
uddf.feats.value_counts()

                                                                                                64473
PronType=Art                                                                                    16413
Gender=Masc|Number=Sing                                                                         14687
Gender=Fem|Number=Sing                                                                           9948
Gender=Masc|Number=Plur                                                                          5725
                                                                                                ...  
Gender=Fem,Masc|HebSource=ConvUncertainHead|Number=Plur|Person=1|Tense=Fut|VerbType=Mod             1
Definite=Cons|Gender=Masc|HebBinyan=HUFAL|Number=Sing|Person=1,2,3|VerbForm=Part|Voice=Pass         1
Gender=Masc|Number=Sing|Person=2|Tense=Fut                                                          1
Gender=Fem,Masc|HebExistential=True|Number=Plur|Person=1|Polarity=Pos|Tense=Fut   

In [266]:
uddf.misc.value_counts()

                                                                                     148003
SpaceAfter=No                                                                         12178
SpaceAfter=No|biose=E-PER|ner_escaped=PER[16]|token_id=2|token_str=ארד                    1
SpaceAfter=No|biose=O|ner_escaped=_|token_id=17|token_str=רשיון                           1
SpaceAfter=No|biose=O|ner_escaped=_|token_id=21|token_str=זה                              1
SpaceAfter=No|biose=O|ner_escaped=_|token_id=13|token_str="                               1
SpaceAfter=No|biose=O|ner_escaped=_|token_id=15|token_str=זרים                            1
SpaceAfter=No|biose=O|ner_escaped=_|token_id=22|token_str="                               1
SpaceAfter=No|biose=O|ner_escaped=_|token_id=21|token_str=בין                             1
SpaceAfter=No|biose=O|ner_escaped=_|token_id=15|token_str=סירבה                           1
SpaceAfter=No|biose=O|ner_escaped=_|token_id=7|token_str=(                      

In [267]:
from functools import lru_cache

plo_map = {
    'ANG': None,
    'DUC': None,
    'EVE': None,
    'FAC': 'LOC',
    'GPE': 'LOC',
    'LOC': 'LOC',
    'ORG': 'ORG',
    'PER': 'PER',
    'WOA': None,
}

nocat_map = {
    'ANG': 'ENT',
    'DUC': 'ENT',
    'EVE': 'ENT',
    'FAC': 'ENT',
    'GPE': 'ENT',
    'LOC': 'ENT',
    'ORG': 'ENT',
    'PER': 'ENT',
    'WOA': 'ENT',
}

def replace_labels(label, label_map):
    if label=='O':
        return 'O'
    else:
        b, c = label.split('-')
        if c in label_map:
            m = label_map[c]
            if m is None:
                return 'O'
            else:
                return b+'-'+m
        else:
            raise KeyError
            

In [268]:
uddf['biose_layer0_plo'] = uddf.biose_layer0.apply(replace_labels, args=(plo_map,))
uddf['biose_layer0_nocat'] = uddf.biose_layer0.apply(replace_labels, args=(nocat_map,))
uddf[['form', 'biose_layer0_plo']].head()

,form,biose_layer0_plo
0,עשרות,O
1,אנשים,O
2,מגיעים,O
3,מ,O
4,תאילנד,S-LOC


In [274]:
'O'.split('-')

['O']

In [276]:
def get_bio_only(label):
    return label.split('-')[0]

def get_ner_cat(label):
    if label == 'O':
        return '_'
    else: 
        return label.split('-')[1]

uddf['ner_cat_plo']    = uddf.biose_layer0_plo.apply(get_ner_cat)
uddf['bio_only_plo']   = uddf.biose_layer0_plo.apply(get_bio_only)
uddf['ner_cat_nocat']  = uddf.biose_layer0_nocat.apply(get_ner_cat)
uddf['bio_only_nocat'] = uddf.biose_layer0_nocat.apply(get_bio_only)
uddf['ner_cat_full']   = uddf.biose_layer0.apply(get_ner_cat)
uddf['bio_only_full']  = uddf.biose_layer0.apply(get_bio_only)

uddf[['ner_cat_plo', 'bio_only_full']].head()

,ner_cat_plo,bio_only_full
0,_,O
1,_,O
2,_,O
3,_,O
4,LOC,S


In [210]:
from collections import OrderedDict
DEFAULT_FIELDS = ['id', 'form', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc']

def sentence_to_conllu_token_list(sent, add_misc=['token_id', 'token_str', 'biose_layer0', 
                                                  'biose_layer1', 'biose_layer2', 'biose_layer3'],
                                  add_feats=None, replace_feats=False, replace_misc=False,
                                  metadata=['sent_id', 'global_sent_id', 'set', 'duplicate_sent_id', 'very_similar_sent_id']):
    metadata = OrderedDict([(m, str(sent[m].iat[0])) for m in metadata])
    
    if add_misc is not None:
        amisc = sent.apply(lambda x: [m+'='+str(x[m]) for m in add_misc], axis=1)
        amisc = amisc.str.join('|')
    if add_feats is not None:
        afeats = sent.apply(lambda x: {m: str(x[m]) for m in add_feats}, axis=1)

    token_morphs = (sent.groupby('token_id').id.agg([min, max])
                 .pipe(lambda x: x[x['max'] - x['min']>0])
                 .assign(token_morphs=lambda x: x['min'].astype(str)+'-'+x['max'].astype(str))
                 .set_index('min')['token_morphs'])   
    token_list = []
    
    for (i, row), token_str, am, af in zip(sent[DEFAULT_FIELDS].iterrows(), sent['token_str'].tolist(), amisc.tolist(), afeats.tolist()):
        token = OrderedDict(row)
        if not replace_feats and type(token['feats']) == str and token['feats']!='_' and  token['feats']!='':
            #token['feats'] = OrderedDict([f.split('=') for f in token['feats'].split('|')])
            feats = OrderedDict()
            for f in token['feats'].split('|'):
                k,v = f.split('=')
                if k not in feats:
                    feats[k] = v
                else:
                    feats[k] = feats[k]+','+v    
            feats.update(af)
        elif af:
            feats = OrderedDict()
            feats.update(af)
        token['feats'] = feats
        
        am = OrderedDict([f.split('=') for f in am.split('|') ])
        if replace_misc or token['misc'] =='_' or token['misc']=='' or token['misc'] is None or token['misc'] is np.nan:
            token['misc'] = am
        elif type(token['misc'])==str:
            token['misc'] = OrderedDict([f.split('=') for f in token['misc'].split('|') ])
            token['misc'].update(am)
        elif type(token['misc'])==OrderedDict:
            am.update(token['misc'])
            token['misc'] = am
            
        if token['id'] in token_morphs:
            tok_row = OrderedDict([(f, '_') for f in DEFAULT_FIELDS])
            id_tup = token_morphs[token['id']].split('-')
            tok_row['id'] = (id_tup[0], '-', id_tup[1])
            tok_row['form'] = token_str
            token_list.append(tok_row)
        
        token_list.append(token)
        
    return token_list, metadata



In [211]:
from conllu.models import TokenList
conllu_sents = [TokenList(*sentence_to_conllu_token_list(uddf[uddf.sent_id==i].fillna('_'), add_misc=['token_id', 'token_str'], add_feats=['biose_layer0_plo'], replace_feats=True)) for i in uddf[uddf.sent_id<10].sent_id.drop_duplicates().tolist()]
print(conllu_sents[0].serialize())

# sent_id = 1
# global_sent_id = 1
# set = dev
# duplicate_sent_id = _
# very_similar_sent_id = _
1	עשרות	עשרות	NUM	NUM	biose_layer0_plo=O	2	nummod	_	token_id=1|token_str=עשרות
2	אנשים	איש	NOUN	NOUN	biose_layer0_plo=O	3	nsubj	_	token_id=2|token_str=אנשים
3	מגיעים	הגיע	VERB	VERB	biose_layer0_plo=O	0	root	_	token_id=3|token_str=מגיעים
4-5	מתאילנד	_	_	_	_	_	_	_	_
4	מ	מ	ADP	ADP	biose_layer0_plo=O	5	case	_	token_id=4|token_str=מתאילנד
5	תאילנד	תאילנד	PROPN	PROPN	biose_layer0_plo=S-LOC	3	obl	_	token_id=4|token_str=מתאילנד
6-7	לישראל	_	_	_	_	_	_	_	_
6	ל	ל	ADP	ADP	biose_layer0_plo=O	7	case	_	token_id=5|token_str=לישראל
7	ישראל	ישראל	PROPN	PROPN	biose_layer0_plo=S-LOC	3	obl	_	token_id=5|token_str=לישראל
8-9	כשהם	_	_	_	_	_	_	_	_
8	כש	כש	SCONJ	SCONJ	biose_layer0_plo=O	10	mark	_	token_id=6|token_str=כשהם
9	הם	הוא	PRON	PRON	biose_layer0_plo=O	10	nsubj	_	token_id=6|token_str=כשהם
10	נרשמים	נרשם	VERB	VERB	biose_layer0_plo=O	3	advcl	_	token_id=7|token_str=נרשמים
11-12	כמתנדבים	_	_	_	_	_	_	_	_
11	כ	כ	A

## TODO:
1. create gold full, plo, nocat serialized conllu
1. choose matching sentences for UD (order + set - train,dev,test) 

## Matching sents (dev, train , test)

In [169]:

def get_sent_ids_lat(path):
    sent_ids = []
    with bz2.open(path, 'rt', encoding='utf-8') as f:
        for line in f:
            if line[0]=='#':
                sid = sent_id_re.match(line)
                if sid:
                    sent_id =  int(sid.group(1))
                sent_ids.append(sent_id)
    return sent_ids

dev_sent_ids = get_sent_ids_lat('../UL_Hebrew-HTB/he_htb-ud-dev.heblex.conllul.bz2')
train_sent_ids = get_sent_ids_lat('../UL_Hebrew-HTB/he_htb-ud-train.heblex.conllul.bz2')
test_sent_ids = get_sent_ids_lat('../UL_Hebrew-HTB/he_htb-ud-test.heblex.conllul.bz2')

            

In [269]:
print(f'dev: {min(dev_sent_ids)}-{max(dev_sent_ids)}')
print(f'train: {min(train_sent_ids)}-{max(train_sent_ids)}')
print(f'test: {min(test_sent_ids)}-{max(test_sent_ids)}')

dev: 1-484
train: 485-5725
test: 5726-6216


In [270]:
uddf.groupby(['set']).sent_id.agg(['min', 'max'])

,min,max
set,,
dev,1,500
test,5502,6216
train,501,5500


In [271]:
def get_set_from_sent_id(gsi):
    if gsi>=min(dev_sent_ids) and gsi<=max(dev_sent_ids):
        return 'dev'
    elif gsi>=min(train_sent_ids) and gsi<=max(train_sent_ids):
        return 'train'
    elif gsi>=min(test_sent_ids) and gsi<=max(test_sent_ids):
        return 'test'
    
uddf['ud_set'] = uddf.sent_id.apply(get_set_from_sent_id)

uddf.groupby(['ud_set']).sent_id.agg(['min', 'max'])

,min,max
ud_set,,
dev,1,484
test,5726,6216
train,485,5725


In [287]:
ud_dev = uddf[uddf.ud_set=='dev']
ud_train = uddf[uddf.ud_set=='train']
ud_test = uddf[uddf.ud_set=='test']

In [214]:
def get_conllu_sents(df, add_feats=['biose_layer0'], replace_feats=False, replace_misc=True):
    conllu_sents = [TokenList(*sentence_to_conllu_token_list(df[df.sent_id==i].fillna('_'), add_misc=['token_id', 'token_str'], add_feats=add_feats, metadata=['sent_id'], replace_feats=replace_feats, replace_misc=replace_misc)) for i in df.sent_id.drop_duplicates().tolist()]
    return conllu_sents



In [218]:
import os

out_folder = 'data/for_amit/gold'
if not os.path.exists(out_folder):
    os.mkdir(out_folder)
    
def write_conllu_file(sent_list, path):
    with open(path, 'w', encoding='utf8') as f: 
        for token_list in sent_list:
            f.write(token_list.serialize())
            
write_conllu_file(plo_only_dev, os.path.join(out_folder, 'he_htb-ud-dev.ner_plo_only.conllu'))

## ONLY

In [219]:
plo_only_dev = get_conllu_sents(ud_dev, add_feats=['biose_layer0_plo'], replace_feats=True)
write_conllu_file(plo_only_dev, os.path.join(out_folder, 'he_htb-ud-dev.ner_plo_only.conllu'))

plo_only_train = get_conllu_sents(ud_train, add_feats=['biose_layer0_plo'], replace_feats=True)
write_conllu_file(plo_only_train, os.path.join(out_folder, 'he_htb-ud-train.ner_plo_only.conllu'))

plo_only_test = get_conllu_sents(ud_test, add_feats=['biose_layer0_plo'], replace_feats=True)
write_conllu_file(plo_only_test, os.path.join(out_folder, 'he_htb-ud-test.ner_plo_only.conllu'))

In [220]:
nocat_only_dev = get_conllu_sents(ud_dev, add_feats=['biose_layer0_nocat'], replace_feats=True)
write_conllu_file(nocat_only_dev, os.path.join(out_folder, 'he_htb-ud-dev.ner_nocat_only.conllu'))

nocat_only_train = get_conllu_sents(ud_train, add_feats=['biose_layer0_nocat'], replace_feats=True)
write_conllu_file(nocat_only_train, os.path.join(out_folder, 'he_htb-ud-train.ner_nocat_only.conllu'))

nocat_only_test = get_conllu_sents(ud_test, add_feats=['biose_layer0_nocat'], replace_feats=True)
write_conllu_file(nocat_only_test, os.path.join(out_folder, 'he_htb-ud-test.ner_nocat_only.conllu'))

In [222]:
full_only_dev = get_conllu_sents(ud_dev, add_feats=['biose_layer0'], replace_feats=True)
write_conllu_file(full_only_dev, os.path.join(out_folder, 'he_htb-ud-dev.ner_full_only.conllu'))

full_only_train = get_conllu_sents(ud_train, add_feats=['biose_layer0'], replace_feats=True)
write_conllu_file(full_only_train, os.path.join(out_folder, 'he_htb-ud-train.ner_full_only.conllu'))

full_only_test = get_conllu_sents(ud_test, add_feats=['biose_layer0'], replace_feats=True)
write_conllu_file(full_only_test, os.path.join(out_folder, 'he_htb-ud-test.ner_full_only.conllu'))

## ADD TO FEATS

In [223]:
plo_dev = get_conllu_sents(ud_dev, add_feats=['biose_layer0_plo'], replace_feats=False)
write_conllu_file(plo_dev, os.path.join(out_folder, 'he_htb-ud-dev.ner_plo.conllu'))

plo_train = get_conllu_sents(ud_train, add_feats=['biose_layer0_plo'], replace_feats=False)
write_conllu_file(plo_train, os.path.join(out_folder, 'he_htb-ud-train.ner_plo.conllu'))

plo_test = get_conllu_sents(ud_test, add_feats=['biose_layer0_plo'], replace_feats=False)
write_conllu_file(plo_test, os.path.join(out_folder, 'he_htb-ud-test.ner_plo.conllu'))

In [224]:
nocat_dev = get_conllu_sents(ud_dev, add_feats=['biose_layer0_nocat'], replace_feats=False)
write_conllu_file(nocat_dev, os.path.join(out_folder, 'he_htb-ud-dev.ner_nocat.conllu'))

nocat_train = get_conllu_sents(ud_train, add_feats=['biose_layer0_nocat'], replace_feats=False)
write_conllu_file(nocat_train, os.path.join(out_folder, 'he_htb-ud-train.ner_nocat.conllu'))

nocat_test = get_conllu_sents(ud_test, add_feats=['biose_layer0_nocat'], replace_feats=False)
write_conllu_file(nocat_test, os.path.join(out_folder, 'he_htb-ud-test.ner_nocat.conllu'))

In [226]:
full_dev = get_conllu_sents(ud_dev, add_feats=['biose_layer0'], replace_feats=False)
write_conllu_file(full_dev, os.path.join(out_folder, 'he_htb-ud-dev.ner_full.conllu'))

full_train = get_conllu_sents(ud_train, add_feats=['biose_layer0'], replace_feats=False)
write_conllu_file(full_train, os.path.join(out_folder, 'he_htb-ud-train.ner_full.conllu'))

full_test = get_conllu_sents(ud_test, add_feats=['biose_layer0'], replace_feats=False)
write_conllu_file(full_test, os.path.join(out_folder, 'he_htb-ud-test.ner_full.conllu'))

## Replace POS

In [300]:
from collections import OrderedDict
DEFAULT_FIELDS = ['id', 'form', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc']

def sentence_to_conllu_token_list_pos(sent, replace_pos = 'biose_layer0_seq',
                                  replace_feats='ner_cat',
                                  metadata=['sent_id', 'global_sent_id', 'set', 'duplicate_sent_id', 'very_similar_sent_id']):
    metadata = OrderedDict([(m, str(sent[m].iat[0])) for m in metadata])

    token_morphs = (sent.groupby('token_id').id.agg([min, max])
                 .pipe(lambda x: x[x['max'] - x['min']>0])
                 .assign(token_morphs=lambda x: x['min'].astype(str)+'-'+x['max'].astype(str))
                 .set_index('min')['token_morphs'])   
    token_list = []
    
    for (i, row), token_str, pos, feats in zip(sent[DEFAULT_FIELDS].iterrows(), sent['token_str'].tolist(), sent[replace_pos].tolist(), sent[replace_feats].tolist()):
        token = OrderedDict(row)
        if feats=='_':
            token['feats'] = '_'
        else:
            token['feats'] = replace_feats+'='+feats
        token['upostag'] = pos
        token['xpostag'] = pos
        token['misc'] = '_'
            
        if token['id'] in token_morphs:
            tok_row = OrderedDict([(f, '_') for f in DEFAULT_FIELDS])
            id_tup = token_morphs[token['id']].split('-')
            tok_row['id'] = (id_tup[0], '-', id_tup[1])
            tok_row['form'] = token_str
            token_list.append(tok_row)
        
        token_list.append(token)
        
    return token_list, metadata



In [302]:
from conllu.models import TokenList
conllu_sents = [TokenList(*sentence_to_conllu_token_list_pos(uddf[uddf.sent_id==i].fillna('_'), replace_pos='bio_only_plo', replace_feats='ner_cat_plo')) for i in uddf[uddf.sent_id<10].sent_id.drop_duplicates().tolist()]
print(conllu_sents[0].serialize())

# sent_id = 1
# global_sent_id = 1
# set = dev
# duplicate_sent_id = _
# very_similar_sent_id = _
1	עשרות	עשרות	O	O	_	2	nummod	_	_
2	אנשים	איש	O	O	_	3	nsubj	_	_
3	מגיעים	הגיע	O	O	_	0	root	_	_
4-5	מתאילנד	_	_	_	_	_	_	_	_
4	מ	מ	O	O	_	5	case	_	_
5	תאילנד	תאילנד	S	S	ner_cat_plo=LOC	3	obl	_	_
6-7	לישראל	_	_	_	_	_	_	_	_
6	ל	ל	O	O	_	7	case	_	_
7	ישראל	ישראל	S	S	ner_cat_plo=LOC	3	obl	_	_
8-9	כשהם	_	_	_	_	_	_	_	_
8	כש	כש	O	O	_	10	mark	_	_
9	הם	הוא	O	O	_	10	nsubj	_	_
10	נרשמים	נרשם	O	O	_	3	advcl	_	_
11-12	כמתנדבים	_	_	_	_	_	_	_	_
11	כ	כ	O	O	_	12	case	_	_
12	מתנדבים	מתנדב	O	O	_	10	obl	_	_
13	,	,	O	O	_	10	punct	_	_
14	אך	אך	O	O	_	16	cc	_	_
15	למעשה	למעשה	O	O	_	16	advmod	_	_
16	משמשים	שימש	O	O	_	10	conj	_	_
17	עובדים	עובד	O	O	_	16	obj	_	_
18	שכירים	שכיר	O	O	_	17	amod	_	_
19	זולים	זול	O	O	_	17	amod	_	_
20	.	.	O	O	_	3	punct	_	_




In [303]:
out_folder = 'data/for_amit/gold_pos'
if not os.path.exists(out_folder):
    os.mkdir(out_folder)
    
def get_conllu_sents_pos(df, replace_pos, replace_feats):
    conllu_sents = [TokenList(*sentence_to_conllu_token_list_pos(df[df.sent_id==i].fillna('_'), metadata=['sent_id'], replace_pos=replace_pos, replace_feats=replace_feats)) for i in df.sent_id.drop_duplicates().tolist()]
    return conllu_sents



In [304]:
plo_only_dev = get_conllu_sents_pos(ud_dev, 'bio_only_plo', 'ner_cat_plo')
write_conllu_file(plo_only_dev, os.path.join(out_folder, 'he_htb-ud-dev.ner_pos_plo.conllu'), )

plo_only_train = get_conllu_sents_pos(ud_train, 'bio_only_plo', 'ner_cat_plo')
write_conllu_file(plo_only_train, os.path.join(out_folder, 'he_htb-ud-train.ner_pos_plo.conllu'))

plo_only_test = get_conllu_sents_pos(ud_test, 'bio_only_plo', 'ner_cat_plo')
write_conllu_file(plo_only_test, os.path.join(out_folder, 'he_htb-ud-test.ner_pos_plo.conllu'))

In [305]:
nocat_only_dev = get_conllu_sents_pos(ud_dev, 'bio_only_nocat', 'ner_cat_nocat')
write_conllu_file(nocat_only_dev, os.path.join(out_folder, 'he_htb-ud-dev.ner_pos_nocat.conllu'))

nocat_only_train = get_conllu_sents_pos(ud_train, 'bio_only_nocat', 'ner_cat_nocat')
write_conllu_file(nocat_only_train, os.path.join(out_folder, 'he_htb-ud-train.ner_pos_nocat.conllu'))

nocat_only_test = get_conllu_sents_pos(ud_test, 'bio_only_nocat', 'ner_cat_nocat')
write_conllu_file(nocat_only_test, os.path.join(out_folder, 'he_htb-ud-test.ner_pos_nocat.conllu'))

In [306]:
full_only_dev = get_conllu_sents_pos(ud_dev, 'bio_only_full', 'ner_cat_full')
write_conllu_file(full_only_dev, os.path.join(out_folder, 'he_htb-ud-dev.ner_pos_full.conllu'))

full_only_train = get_conllu_sents_pos(ud_train, 'bio_only_full', 'ner_cat_full')
write_conllu_file(full_only_train, os.path.join(out_folder, 'he_htb-ud-train.ner_pos_full.conllu'))

full_only_test = get_conllu_sents_pos(ud_test, 'bio_only_full', 'ner_cat_full')
write_conllu_file(full_only_test, os.path.join(out_folder, 'he_htb-ud-test.ner_pos_full.conllu'))